## 🎵 KaraFan

The BEST music separation model with help of **A.I** ... to my ears ! 👂👂

#### 📝 The Story of this project, Explanations, and Thanks are on my  &nbsp;[![GitHub](https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&labelColor=333&logoColor=white&color=ff0080)](https://github.com/Captain-FLAM/KaraFan)  &nbsp;page.

**R.T.F.R.M** (Read The Fucking READ ME) --> Home page of this GitHub , it's worth it ! 😁

If you want to encourage me to give more time to improve this project :

[![](https://img.buymeacoffee.com/button-api/?text=Buy%20me%20a%20Coffee%20for%20Morning&emoji=:☕&slug=Captain_FLAM&button_colour=c69d8a&font_colour=000000&font_family=Arial&outline_colour=000000&coffee_colour=FFDD00)](https://www.buymeacoffee.com/Captain_FLAM) &nbsp;
[![](https://img.buymeacoffee.com/button-api/?text=Buy%20me%20a%20Beer%20for%20the%20Night&emoji=🍺&slug=Captain_FLAM&button_colour=FFDD00&font_colour=000000&font_family=Arial&outline_colour=000000&coffee_colour=ffffff)](https://www.buymeacoffee.com/Captain_FLAM)  
[![](https://img.buymeacoffee.com/button-api/?text=Become%20my%20Best%20Friend%20....%20&emoji=❤️&slug=Captain_FLAM&button_colour=f4007a&font_colour=ffffff&font_family=Arial&outline_colour=ffffff&coffee_colour=FFDD00)](https://www.buymeacoffee.com/Captain_FLAM/membership) &nbsp;
[![](https://img.buymeacoffee.com/button-api/?text=I%20also%20have%20a%20Dreams%20List%20%3F&emoji=🎁&slug=Captain_FLAM&button_colour=ddd0ca&font_colour=000000&font_family=Arial&outline_colour=000000&coffee_colour=FFDD00)](https://www.buymeacoffee.com/Captain_FLAM/wishlist)

You can become one of my patrons :

[![Patreon](https://img.shields.io/badge/Donate-Patreon-red.svg?style=for-the-badge)](https://www.patreon.com/Captain_FLAM/membership)

And if you have some Bitcoins (or cryptos) that you don't know what to do with,  
click here to send me an email, I'll give you my wallet address :

### [![Bitcoin](https://img.shields.io/badge/Donate-Bitcoin-green.svg?style=for-the-badge)](mailto:Captain_FLAM@gmx.fr) 😍

You can join all of us on this  [![Discord](images/discord.svg) Discord server](https://discord.com/channels/708579735583588363/887455924845944873) !

### 📜 LICENSE

#### &copy; Captain FLAM - 2023 - M.I.T license

That means that if you want to use any part of my code in your projects, **YOU HAVE TO** let this copyright in place !!  
Copyright is not a shame, it's a way to say thanks to the authors for their work !

<a class="goog-tab-bar-end" href="images/KaraFan.png" style="float:right;"><img src="images/KaraFan_mini.png" style="position:absolute;top:100px;right:50px;" alt="KaraFan"></a>

In [1]:
import os, sys, subprocess

#****************************************************************
# If you still have GPU memory errors, you can try to
# uncomment the following line, but I'm not sure it will work...
# Anyway, following the Nvidia documentation,
# they recommend to set this environment variable to 512 MB :

# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:512"
#****************************************************************

Repo_url = "https://github.com/Captain-FLAM/KaraFan"

Gdrive = ""; isColab = False; Fresh = False
try:
	# Are we on Google Colab ?
	from google.colab import drive
	isColab = True
	Gdrive  = "/content/Gdrive/MyDrive"
	drive.mount("/content/Gdrive", timeout_ms=10000)  # 10 seconds
except:
	# We are on a PC : Get the current path and remove last part (KaraFan)
	Gdrive = os.getcwd().replace("KaraFan","").rstrip(os.path.sep)

if isColab and not os.path.exists(Gdrive):
	print("****  You have denied access to your Google Drive !  ****")
	exit(0)

Project_path = os.path.join(Gdrive, "KaraFan")

if not os.path.exists(Project_path):
	print('Cloning "KaraFan" Repository...')
	Fresh = True
	os.chdir(Gdrive)
	try:
		subprocess.run(["git", "clone", f"{Repo_url}.git"], text=True, capture_output=True, check=True)
	except subprocess.CalledProcessError as e:
		print("Error during Cloning :\n" + e.stderr + "\n" + e.stdout)
		exit(1)

if os.path.exists(Project_path):
	# Mandatory to import App modules
	sys.path.insert(0, Project_path)

	import App.gui
else:
	print('"KaraFan" is NOT installed !')
	exit(1)

# 😎 This is it !

In [2]:
App.gui.Run(Gdrive, isColab, Fresh)

Connection error while trying to fetch version : HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /Captain-FLAM/KaraFan/master/App/__init__.py (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002831A494640>: Failed to resolve 'raw.githubusercontent.com' ([Errno 11001] getaddrinfo failed)"))
